In [2]:
from pyomo.environ import *
import numpy as np

# Model
model = ConcreteModel()

# Sets
T = range(1, 11)  # Time periods (1 to 10)
model.T = Set(initialize=T)
model.T_lagged = Set(initialize=[t for t in T if t > 2])  # Periods with transport completion
model.Locations = Set(initialize=['A', 'B'])

# Parameters
model.transport_lag = Param(initialize=2)  # Transport lag
transport_cost = 10  # Cost per unit transported
holding_cost = 1  # Holding cost per unit
initial_inventory = {'A': 500, 'B': 50}
forecast_horizon = 3  # Forecast for 3 periods ahead
true_demand = {t: np.random.randint(80, 120) for t in T}  # Simulated actual demand

# Variables
model.Inventory = Var(model.Locations, model.T, domain=NonNegativeReals)
model.Shipped = Var(model.T, domain=NonNegativeReals)
model.ForecastedDemand = Param(model.T, mutable=True, initialize=0)

# Objective: Minimize total costs
def obj_expression(model):
    holding = sum(holding_cost * model.Inventory[loc, t] for loc in model.Locations for t in model.T)
    transport = sum(transport_cost * model.Shipped[t] for t in model.T)
    return holding + transport

model.Objective = Objective(rule=obj_expression, sense=minimize)

# Constraints
# Inventory balance at Location A
def inventory_balance_a(model, t):
    if t == 1:
        return model.Inventory['A', t] == initial_inventory['A'] - model.Shipped[t]
    return model.Inventory['A', t] == model.Inventory['A', t-1] - model.Shipped[t]

model.InventoryBalanceA = Constraint(model.T, rule=inventory_balance_a)

# Inventory balance at Location B
def inventory_balance_b(model, t):
    if t == 1:
        return model.Inventory['B', t] == initial_inventory['B']
    elif t <= model.transport_lag:
        return model.Inventory['B', t] == model.Inventory['B', t-1]
    else:
        return model.Inventory['B', t] == model.Inventory['B', t-1] + model.Shipped[t - model.transport_lag]

model.InventoryBalanceB = Constraint(model.T, rule=inventory_balance_b)

# Demand fulfillment at Location B
def demand_fulfillment(model, t):
    if t in model.T:
        return model.Inventory['B', t] >= model.ForecastedDemand[t]
    return Constraint.Skip

model.DemandFulfillment = Constraint(model.T, rule=demand_fulfillment)

# Simulation loop
results = []
current_inventory_a = initial_inventory['A']
current_inventory_b = initial_inventory['B']

for t in T:
    # Update forecasted demand
    for k in range(t, min(t + forecast_horizon, max(T) + 1)):
        model.ForecastedDemand[k] = true_demand[k]  # Replace with a forecast function if available
    
    # Solve the model
    solver = SolverFactory('gurobi')  # Use a solver like Gurobi if available
    solver.solve(model)
    
    # Record results for current period
    shipped = model.Shipped[t].value
    inventory_b = model.Inventory['B', t].value
    results.append((t, shipped, inventory_b))
    
    # Update current inventory for simulation
    current_inventory_a -= shipped if shipped else 0
    current_inventory_b = inventory_b if inventory_b else 0

# Display results
print("Period | Shipped | Inventory at B")
for t, shipped, inventory_b in results:
    print(f"{t:6} | {shipped:7.2f} | {inventory_b:14.2f}")


model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.name="unknown";
    - termination condition: infeasible
    - message from solver: Model was proven to be infeasible.
model.na

TypeError: unsupported format string passed to NoneType.__format__

In [3]:
model.pprint()

3 Set Declarations
    Locations : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'A', 'B'}
    T : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    T_lagged : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {3, 4, 5, 6, 7, 8, 9, 10}

2 Param Declarations
    ForecastedDemand : Size=10, Index=T, Domain=Any, Default=None, Mutable=True
        Key : Value
          1 :    82
          2 :    84
          3 :   109
          4 :    84
          5 :    83
          6 :    95
          7 :   104
          8 :    92
          9 :   115
         10 :    95
    transport_lag : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     2

2 Var Declarations
    Inventory : Size=20, Index=Locati